In [47]:
import pandas as pd

In [48]:
df = pd.read_csv('dirty_cafe_sales.csv')
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


In [49]:
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [50]:
df = df.replace(['ERROR','UNKNOWN'],pd.NA)

In [51]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [52]:
numerical_columns = ['quantity','price_per_unit','total_spent']
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col],errors='coerce')

In [53]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'],errors='coerce')

In [54]:
df[df['price_per_unit'].isnull()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
56,TXN_3578141,Cake,5.0,NaN,15.0,NaN,Takeaway,2023-06-27
65,TXN_4987129,Sandwich,3.0,NaN,NaN,NaN,In-store,2023-10-20
68,TXN_8427104,Salad,2.0,NaN,10.0,NaN,In-store,2023-10-27
85,TXN_8035512,Tea,3.0,NaN,4.5,Cash,<NA>,2023-10-29
104,TXN_7447872,Juice,2.0,NaN,6.0,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...
9924,TXN_5981429,Juice,2.0,NaN,6.0,Digital Wallet,NaN,2023-12-24
9926,TXN_2464706,Cake,4.0,NaN,12.0,Digital Wallet,Takeaway,2023-11-09
9961,TXN_2153100,Tea,2.0,NaN,3.0,Cash,NaN,2023-12-29
9996,TXN_9659401,NaN,3.0,NaN,3.0,Digital Wallet,NaN,2023-06-02


In [55]:
df = df[~(((df['item'].isnull() & df['price_per_unit'].isnull()) & (df['quantity'].isnull() | df['total_spent'].isnull())) | (df['quantity'].isnull() & df['total_spent'].isnull()))]

In [56]:
filter = (df['price_per_unit'].isnull() & df['total_spent'].notnull() & df['quantity'].notnull())

In [57]:
df.loc[filter,'price_per_unit'] = df.loc[filter,'total_spent']/df.loc[filter,'quantity']

In [58]:
df['price_per_unit'] = df['price_per_unit'].fillna(df.groupby('item')['price_per_unit'].transform('max'))

In [59]:
df.groupby('item')['price_per_unit'].max()

item
Cake        3.0
Coffee      2.0
Cookie      1.0
Juice       3.0
Salad       5.0
Sandwich    4.0
Smoothie    4.0
Tea         1.5
Name: price_per_unit, dtype: float64

In [60]:
mapping_item = df.groupby('price_per_unit')['item'].apply(lambda x:x.mode().iloc[0]).to_dict()
mapping_item

{1.0: 'Cookie',
 1.5: 'Tea',
 2.0: 'Coffee',
 3.0: 'Juice',
 4.0: 'Sandwich',
 5.0: 'Salad'}

In [61]:
df['item'] = df['item'].fillna(df['price_per_unit'].map(mapping_item))

In [62]:
df['quantity'] = df['quantity'].fillna(df['total_spent']/df['price_per_unit'])

In [63]:
df['total_spent'] = df['total_spent'].fillna(df['quantity'] * df['price_per_unit'])

In [64]:
df['payment_method'] = df['payment_method'].fillna('Unknown')

In [65]:
df['location'] = df['location'].fillna('Unknown')

In [66]:
df.isnull().sum()

transaction_id        0
item                  0
quantity              0
price_per_unit        0
total_spent           0
payment_method        0
location              0
transaction_date    460
dtype: int64

In [70]:
df['id'] = df['transaction_id'].str.split('_').str[1]

In [75]:
df = df.sort_values('id').drop(columns='id')

In [77]:
df['date_diff'] = df['transaction_date'].diff().dt.days

In [81]:
df['red_flag'] = (df['date_diff'] < 0).astype(int)

In [83]:
df['transaction_date'] = df['transaction_date'].fillna(df['transaction_date'].ffill())

In [86]:
df.to_csv('cleaneddata.csv')